<a href="https://colab.research.google.com/github/ykato27/AutoML-AutoGluon/blob/main/AutoGluon_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AutoGluonとは
mxnet製の[Auto MLライブラリ](https://autogluon.mxnet.io/)

In [1]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install --upgrade "mxnet<2.0.0"
!pip install --pre autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

# trainのデータセットの2割をモデル学習時のバリデーションデータとして利用する
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [6]:
label = 'MEDV'
feature_names = x_train.columns
train_data = x_train.copy()
train_data[label] = y_train
test_data = x_test.copy()
test_data[label] = y_test

display(train_data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
99,0.06860,0.0,2.89,0.0,0.445,7.416,62.5,3.4952,2.0,276.0,18.0,396.90,6.19,33.2
467,4.42228,0.0,18.10,0.0,0.584,6.003,94.5,2.5403,24.0,666.0,20.2,331.29,21.32,19.1
356,8.98296,0.0,18.10,1.0,0.770,6.212,97.4,2.1222,24.0,666.0,20.2,377.73,17.60,17.8
335,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,21.1
474,8.05579,0.0,18.10,0.0,0.584,5.427,95.4,2.4298,24.0,666.0,20.2,352.58,18.14,13.8


In [7]:
display(test_data.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
305,0.05479,33.0,2.18,0.0,0.472,6.616,58.1,3.3700,7.0,222.0,18.4,393.36,8.93,28.4
193,0.02187,60.0,2.93,0.0,0.401,6.800,9.9,6.2196,1.0,265.0,15.6,393.37,5.03,31.1
65,0.03584,80.0,3.37,0.0,0.398,6.290,17.8,6.6115,4.0,337.0,16.1,396.90,4.67,23.5
349,0.02899,40.0,1.25,0.0,0.429,6.939,34.5,8.7921,1.0,335.0,19.7,389.85,5.89,26.6
151,1.49632,0.0,19.58,0.0,0.871,5.404,100.0,1.5916,5.0,403.0,14.7,341.60,13.28,19.6


### オプション無し

In [8]:
predictor = TabularPredictor(label='MEDV', problem_type='regression').fit(train_data, time_limit=None) 
leaderboard = predictor.leaderboard(test_data)
leaderboard

No path specified. Models will be saved in: "AutogluonModels/ag-20210624_010259/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210624_010259/"
AutoGluon Version:  0.2.1b20210623
Train Data Rows:    323
Train Data Columns: 13
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12742.27 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX

[1000]	train_set's rmse: 1.6496	valid_set's rmse: 3.04659
[2000]	train_set's rmse: 1.09009	valid_set's rmse: 2.83094
[3000]	train_set's rmse: 0.797298	valid_set's rmse: 2.78525
[4000]	train_set's rmse: 0.626035	valid_set's rmse: 2.76701


	-2.7652	 = Validation root_mean_squared_error score
	1.47s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	train_set's rmse: 0.533176	valid_set's rmse: 2.77543


	-2.761	 = Validation root_mean_squared_error score
	0.63s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-4.3411	 = Validation root_mean_squared_error score
	0.74s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-3.8642	 = Validation root_mean_squared_error score
	1.93s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-3.7804	 = Validation root_mean_squared_error score
	0.63s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-4.3285	 = Validation root_mean_squared_error score
	1.28s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-4.3804	 = Validation root_mean_squared_error score
	0.46s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-3.29	 = Validation root_mean_squared_error score
	4.43s	 = Training runtime
	0.13s	 = Validation runtime
Fitting model: LightGBMLarge ...
	-3.5454	 = Valida

                  model  score_test  score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        NeuralNetMXNet   -3.279707  -3.289998        0.146786       0.129863  4.426901                 0.146786                0.129863           4.426901            1       True         10
1              CatBoost   -3.404312  -3.864178        0.003371       0.001986  1.932894                 0.003371                0.001986           1.932894            1       True          6
2               XGBoost   -3.751685  -4.380429        0.061539       0.005029  0.464490                 0.061539                0.005029           0.464490            1       True          9
3   WeightedEnsemble_L2   -3.779791  -2.677469        0.246044       0.158490  6.981989                 0.005026                0.000684           0.451506            2       True         12
4         ExtraTreesMSE   -3.903037  -3.78037

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetMXNet,-3.279707,-3.289998,0.146786,0.129863,4.426901,0.146786,0.129863,4.426901,1,True,10
1,CatBoost,-3.404312,-3.864178,0.003371,0.001986,1.932894,0.003371,0.001986,1.932894,1,True,6
2,XGBoost,-3.751685,-4.380429,0.061539,0.005029,0.464490,0.061539,0.005029,0.464490,1,True,9
3,WeightedEnsemble_L2,-3.779791,-2.677469,0.246044,0.158490,6.981989,0.005026,0.000684,0.451506,2,True,12
4,ExtraTreesMSE,-3.903037,-3.780375,0.113951,0.103447,0.628455,0.113951,0.103447,0.628455,1,True,7
5,RandomForestMSE,-3.987919,-4.341120,0.117919,0.103149,0.744809,0.117919,0.103149,0.744809,1,True,5
6,LightGBMXT,-3.988189,-2.765241,0.065881,0.020281,1.472048,0.065881,0.020281,1.472048,1,True,3
7,LightGBM,-4.259504,-2.761029,0.028351,0.007663,0.631535,0.028351,0.007663,0.631535,1,True,4
8,LightGBMLarge,-4.320950,-3.545373,0.015574,0.010960,0.488189,0.015574,0.010960,0.488189,1,True,11
9,NeuralNetFastAI,-4.511627,-4.328461,0.022405,0.016724,1.278340,0.022405,0.016724,1.278340,1,True,8


In [9]:
y_pred = predictor.predict(test_data)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -3.77979101112008
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -3.77979101112008,
    "mean_squared_error": -14.286820087744163,
    "mean_absolute_error": -2.557449280981924,
    "r2": 0.8633895717302121,
    "pearsonr": 0.9332392557691281,
    "median_absolute_error": -1.7555456161499023
}


Predictions:   305    29.202087
193    28.663527
65     26.790657
349    23.962158
151    18.179682
         ...    
208    23.886160
174    25.854280
108    19.739082
242    20.812876
102    17.641972
Name: MEDV, Length: 102, dtype: float32


### 高精度モデル

In [10]:
predictor = TabularPredictor(label='MEDV', problem_type='regression').fit(train_data, presets='best_quality', time_limit=None) # 高精度モデル
leaderboard = predictor.leaderboard(test_data)
leaderboard

No path specified. Models will be saved in: "AutogluonModels/ag-20210624_010315/"
Presets specified: ['best_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210624_010315/"
AutoGluon Version:  0.2.1b20210623
Train Data Rows:    323
Train Data Columns: 13
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12624.38 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | 

[1000]	train_set's rmse: 1.65198	valid_set's rmse: 3.28447
[1000]	train_set's rmse: 1.6068	valid_set's rmse: 3.24237
[1000]	train_set's rmse: 1.45055	valid_set's rmse: 4.54294
[2000]	train_set's rmse: 0.954812	valid_set's rmse: 4.37411
[3000]	train_set's rmse: 0.687997	valid_set's rmse: 4.29424
[4000]	train_set's rmse: 0.529676	valid_set's rmse: 4.25651
[5000]	train_set's rmse: 0.425019	valid_set's rmse: 4.2418
[6000]	train_set's rmse: 0.352708	valid_set's rmse: 4.23501
[7000]	train_set's rmse: 0.300485	valid_set's rmse: 4.22532
[8000]	train_set's rmse: 0.26141	valid_set's rmse: 4.22184
[9000]	train_set's rmse: 0.232762	valid_set's rmse: 4.21831
[10000]	train_set's rmse: 0.210759	valid_set's rmse: 4.21411
[1000]	train_set's rmse: 1.58977	valid_set's rmse: 2.93582
[2000]	train_set's rmse: 1.03558	valid_set's rmse: 2.89765
[3000]	train_set's rmse: 0.742461	valid_set's rmse: 2.88636


	-3.3536	 = Validation root_mean_squared_error score
	4.9s	 = Training runtime
	0.07s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...


[1000]	train_set's rmse: 0.375455	valid_set's rmse: 4.5333
[2000]	train_set's rmse: 0.149146	valid_set's rmse: 4.50421


	-3.4233	 = Validation root_mean_squared_error score
	1.84s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-3.5678	 = Validation root_mean_squared_error score
	0.82s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	-2.9904	 = Validation root_mean_squared_error score
	8.48s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-3.3225	 = Validation root_mean_squared_error score
	0.63s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	-3.8441	 = Validation root_mean_squared_error score
	3.81s	 = Training runtime
	0.07s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	-3.2736	 = Validation root_mean_squared_error score
	1.48s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetMXNet_BAG_L1 ...
	-3.3027	 = Validation root_mean_squared_error score
	15.56s	 = Training runtime
	0.66s	 = Validation runtime
Fit

[1000]	train_set's rmse: 0.0261636	valid_set's rmse: 4.44075


	-3.4209	 = Validation root_mean_squared_error score
	3.76s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-2.9054	 = Validation root_mean_squared_error score
	0.49s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 45.45s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20210624_010315/")


                     model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    NeuralNetMXNet_BAG_L1   -3.270097  -3.302694        0.727729       0.664216  15.561107                 0.727729                0.664216          15.561107            1       True         10
1      WeightedEnsemble_L2   -3.490292  -2.905403        1.389280       0.865838  30.915151                 0.005956                0.000641           0.486877            2       True         12
2          CatBoost_BAG_L1   -3.628917  -2.990442        0.012769       0.011595   8.484338                 0.012769                0.011595           8.484338            1       True          6
3     ExtraTreesMSE_BAG_L1   -3.767122  -3.322457        0.121114       0.094917   0.634943                 0.121114                0.094917           0.634943            1       True          7
4           XGBoost_BAG_L

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetMXNet_BAG_L1,-3.270097,-3.302694,0.727729,0.664216,15.561107,0.727729,0.664216,15.561107,1,True,10
1,WeightedEnsemble_L2,-3.490292,-2.905403,1.389280,0.865838,30.915151,0.005956,0.000641,0.486877,2,True,12
2,CatBoost_BAG_L1,-3.628917,-2.990442,0.012769,0.011595,8.484338,0.012769,0.011595,8.484338,1,True,6
3,ExtraTreesMSE_BAG_L1,-3.767122,-3.322457,0.121114,0.094917,0.634943,0.121114,0.094917,0.634943,1,True,7
4,XGBoost_BAG_L1,-3.804868,-3.273567,0.230889,0.022142,1.475436,0.230889,0.022142,1.475436,1,True,9
5,LightGBMLarge_BAG_L1,-3.881084,-3.420865,0.142810,0.031747,3.764165,0.142810,0.031747,3.764165,1,True,11
6,RandomForestMSE_BAG_L1,-3.969013,-3.567804,0.119813,0.104563,0.824574,0.119813,0.104563,0.824574,1,True,5
7,LightGBMXT_BAG_L1,-4.041303,-3.353605,0.308711,0.065277,4.899763,0.308711,0.065277,4.899763,1,True,3
8,NeuralNetFastAI_BAG_L1,-4.134344,-3.844150,0.112224,0.071674,3.814726,0.112224,0.071674,3.814726,1,True,8
9,LightGBM_BAG_L1,-4.318825,-3.423332,0.074163,0.024385,1.835708,0.074163,0.024385,1.835708,1,True,4


In [11]:
y_pred = predictor.predict(test_data)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -3.490291712177086
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -3.490291712177086,
    "mean_squared_error": -12.182136236092052,
    "mean_absolute_error": -2.486019461762671,
    "r2": 0.8835145372985369,
    "pearsonr": 0.9483258095162933,
    "median_absolute_error": -1.5435970306396474
}


Predictions:   305    26.497679
193    30.061060
65     26.624653
349    27.348925
151    18.352953
         ...    
208    22.604292
174    22.438900
108    20.307053
242    20.654324
102    19.975018
Name: MEDV, Length: 102, dtype: float32


### Multi-layer stack ensemblingを実行

In [12]:
predictor = TabularPredictor(label='MEDV', problem_type='regression').fit(train_data, auto_stack=True, time_limit=None) # Multi-layer stack ensemblingを有効化
leaderboard = predictor.leaderboard(test_data)
leaderboard

No path specified. Models will be saved in: "AutogluonModels/ag-20210624_010404/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210624_010404/"
AutoGluon Version:  0.2.1b20210623
Train Data Rows:    323
Train Data Columns: 13
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12602.92 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX

[1000]	train_set's rmse: 1.65198	valid_set's rmse: 3.28447
[1000]	train_set's rmse: 1.6068	valid_set's rmse: 3.24237
[1000]	train_set's rmse: 1.45055	valid_set's rmse: 4.54294
[2000]	train_set's rmse: 0.954812	valid_set's rmse: 4.37411
[3000]	train_set's rmse: 0.687997	valid_set's rmse: 4.29424
[4000]	train_set's rmse: 0.529676	valid_set's rmse: 4.25651
[5000]	train_set's rmse: 0.425019	valid_set's rmse: 4.2418
[6000]	train_set's rmse: 0.352708	valid_set's rmse: 4.23501
[7000]	train_set's rmse: 0.300485	valid_set's rmse: 4.22532
[8000]	train_set's rmse: 0.26141	valid_set's rmse: 4.22184
[9000]	train_set's rmse: 0.232762	valid_set's rmse: 4.21831
[10000]	train_set's rmse: 0.210759	valid_set's rmse: 4.21411
[1000]	train_set's rmse: 1.58977	valid_set's rmse: 2.93582
[2000]	train_set's rmse: 1.03558	valid_set's rmse: 2.89765
[3000]	train_set's rmse: 0.742461	valid_set's rmse: 2.88636


	-3.3536	 = Validation root_mean_squared_error score
	4.88s	 = Training runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...


[1000]	train_set's rmse: 0.375455	valid_set's rmse: 4.5333
[2000]	train_set's rmse: 0.149146	valid_set's rmse: 4.50421


	-3.4233	 = Validation root_mean_squared_error score
	1.83s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-3.5678	 = Validation root_mean_squared_error score
	0.83s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	-2.9904	 = Validation root_mean_squared_error score
	8.51s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-3.3225	 = Validation root_mean_squared_error score
	0.63s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	-4.0129	 = Validation root_mean_squared_error score
	3.87s	 = Training runtime
	0.07s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	-3.2736	 = Validation root_mean_squared_error score
	1.47s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetMXNet_BAG_L1 ...
	-3.3959	 = Validation root_mean_squared_error score
	17.05s	 = Training runtime
	0.67s	 = Validation runtime
Fit

[1000]	train_set's rmse: 0.0261636	valid_set's rmse: 4.44075


	-3.4209	 = Validation root_mean_squared_error score
	3.83s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-2.9128	 = Validation root_mean_squared_error score
	0.48s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 47.13s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20210624_010404/")


                     model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    NeuralNetMXNet_BAG_L1   -3.127481  -3.395934        0.725614       0.674453  17.054291                 0.725614                0.674453          17.054291            1       True         10
1      WeightedEnsemble_L2   -3.490728  -2.912762        1.393802       0.889634  32.410575                 0.005035                0.000592           0.477378            2       True         12
2          CatBoost_BAG_L1   -3.628917  -2.990442        0.013572       0.011315   8.514009                 0.013572                0.011315           8.514009            1       True          6
3     ExtraTreesMSE_BAG_L1   -3.767122  -3.322457        0.120358       0.103151   0.627070                 0.120358                0.103151           0.627070            1       True          7
4           XGBoost_BAG_L

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetMXNet_BAG_L1,-3.127481,-3.395934,0.725614,0.674453,17.054291,0.725614,0.674453,17.054291,1,True,10
1,WeightedEnsemble_L2,-3.490728,-2.912762,1.393802,0.889634,32.410575,0.005035,0.000592,0.477378,2,True,12
2,CatBoost_BAG_L1,-3.628917,-2.990442,0.013572,0.011315,8.514009,0.013572,0.011315,8.514009,1,True,6
3,ExtraTreesMSE_BAG_L1,-3.767122,-3.322457,0.120358,0.103151,0.627070,0.120358,0.103151,0.627070,1,True,7
4,XGBoost_BAG_L1,-3.804868,-3.273567,0.241793,0.020129,1.469403,0.241793,0.020129,1.469403,1,True,9
5,LightGBMLarge_BAG_L1,-3.881084,-3.420865,0.148538,0.032421,3.825283,0.148538,0.032421,3.825283,1,True,11
6,RandomForestMSE_BAG_L1,-3.969013,-3.567804,0.121174,0.093792,0.829196,0.121174,0.093792,0.829196,1,True,5
7,LightGBMXT_BAG_L1,-4.041303,-3.353605,0.304579,0.081151,4.884279,0.304579,0.081151,4.884279,1,True,3
8,NeuralNetFastAI_BAG_L1,-4.190221,-4.012885,0.102822,0.071891,3.874168,0.102822,0.071891,3.874168,1,True,8
9,LightGBM_BAG_L1,-4.318825,-3.423332,0.073234,0.025126,1.828697,0.073234,0.025126,1.828697,1,True,4


In [13]:
print(f"Quantile-regression evaluated using metric = {predictor.eval_metric}")

Quantile-regression evaluated using metric = root_mean_squared_error


In [14]:
y_pred = predictor.predict(test_data)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -3.4907276612410447
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -3.4907276612410447,
    "mean_squared_error": -12.185179604953369,
    "mean_absolute_error": -2.4780471465166873,
    "r2": 0.88348543663646,
    "pearsonr": 0.9477746226845486,
    "median_absolute_error": -1.546570587158202
}


Predictions:   305    26.356022
193    29.763405
65     26.639931
349    27.243097
151    18.410011
         ...    
208    22.511509
174    22.663836
108    20.428484
242    20.668835
102    19.856188
Name: MEDV, Length: 102, dtype: float32
